In [ ]:
# Import necessary packages
import pandas as pd
from pyCompare import blandAltman
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from pylab import savefig

In [ ]:
# Define important variables
df = pd.read_csv('predicted_IC50_for_concordance.csv')
name_list = list(df.columns)
name_list = list(df.columns[1:])
ECFP_list = name_list[:5]
Other_list = name_list[5:]

In [ ]:
# Generate Bland-Altman Plots
for a in ECFP_list:
    for b in Other_list:
        plt.rcParams.update({'font.size': 18})
        blandAltman(df[f'{a}'], df[f'{b}'], dpi=300, figureSize=(12,5), figureFormat='png', savePath=f'blandAltmanPlots/{a}_vs_{b}.png')

In [ ]:
# Define important variables and functions, and mask insignificant correlation
target_id = [
    'CHEMBL238', 
    'CHEMBL218', 
    'CHEMBL3650', 
    'CHEMBL220', 
    'CHEMBL248', 
]
df = df.drop('cmpdname', axis=1)
def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = stats.pearsonr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

p_values = corr_sig(df)
mask = np.invert(np.tril(p_values<0.05))

In [ ]:
# Generate Heat Maps
for t in target_id:
    new_df = df.filter(regex=f'{t}')
    # Plotting with significance filter
    corr = new_df.corr()                            # get correlation
    p_values = corr_sig(new_df)                     # get p-Value
    mask = np.invert(np.tril(p_values<0.05))    # mask - only get significant corr

    sns.set(font_scale=1.6)
    heatmap = sns.heatmap(new_df.corr(), mask=mask, annot=True, cmap='rocket')
    figure = heatmap.get_figure()
    figure.savefig(f'heatmaps/png/{t}.png', format = 'png', dpi=600, bbox_inches='tight')
    figure.clf()